<a href="https://colab.research.google.com/github/constantinpape/training-deep-learning-models-for-vison/blob/master/day1/3_multi_layer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-layer Perceptron on CIFAR10

Based on the previous exercise, we will train a multi-layer network, also known as multi-layer perceptron. In this architecture, we will have 'hidden layers', i.e. layers that receive input not from the input directly but from previous layers in the network and that are not directly observed in the output.

## Preperation

In [ ]:
# load tensorboard extension
%load_ext tensorboard

In [ ]:
# import torch and other libraries
import os
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from tqdm import trange

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter

In [ ]:
!pip install cifar2png

In [ ]:
# check if we have gpu support
# colab offers free gpus, however they are not activated by default.
# to activate the gpu, go to 'Runtime->Change runtime type'. 
# Then select 'GPU' in 'Hardware accelerator' and click 'Save'
have_gpu = torch.cuda.is_available()
# we need to define the device for torch, yadda yadda
if have_gpu:
    print("GPU is available")
    device = torch.device('cuda')
else:
    print("GPU is not available, training will run on the CPU")
    device = torch.device('cpu')

In [ ]:
# run this in google colab to get the utils.py file
!wget https://raw.githubusercontent.com/constantinpape/training-deep-learning-models-for-vison/master/day1/utils.py 

In [ ]:
# we will reuse the training function, validation function and
# data preparation from the previous notebook
import utils

In [ ]:
cifar_dir = './cifar10'
!cifar2png cifar10 cifar10

In [ ]:
categories = os.listdir('./cifar10/train')
categories.sort()

## Multi-layer perceptron

In this execise, we go from a single layer network to a network with multiple layers, also known as multi-layer perceptron, or MLP.
We still use fully connected layers (`nn.Linear`), i.e. each neuron in a given layer receives input from all neurons in the previous layer.

Imporantly, we apply a non-linearity to all layer outputs.
Otherwise, the layers could be collapsed into a single layer by matrix multiplication.

In [ ]:
class MLP(nn.Module):
    def __init__(self, n_pixels, n_classes):
        super().__init__()
        self.n_pixels = n_pixels
        self.n_classes = n_classes
        
        # here, we define the structure of the MLP.
        # it's imporant that we use a non-linearity after each 
        # fully connected layer! Here we use the rectified linear
        # unit, short ReLu
        self.layers = nn.Sequential(
            nn.Linear(n_pixels, 400),
            nn.ReLU(),
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, n_classes),
            nn.LogSoftmax(dim=1)
        )
    
    def forward(self, x):
        x = x.view(-1, self.n_pixels)
        x = self.layers(x)
        return x

In [ ]:
# instantiate the model
model = MLP(3072, 10)
model.to(device)

In [ ]:
# get training and validation data
train_dataset, val_dataset = utils.make_cifar_datasets(cifar_dir)

In [ ]:
# instantiate loaders, loss, optimizer and tensorboard

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=25)

optimizer = Adam(model.parameters(), lr=1.e-3)

loss_function = nn.NLLLoss()
loss_function.to(device)

tb_logger = SummaryWriter('runs/log_mlp')
%tensorboard --logdir runs

In [ ]:
# train for a couple of epochs
n_epochs = 4
for epoch in trange(n_epochs):
    utils.train(model, train_loader, loss_function, optimizer,
                device, epoch, tb_logger=tb_logger)
    step = (epoch + 1) * len(train_loader)
    utils.validate(model, val_loader, loss_function,
                   device, step,
                   tb_logger=tb_logger)

In [ ]:
# evaluate the model on test data
test_dataset = utils.make_cifar_test_dataset(cifar_dir)
test_loader = DataLoader(test_dataset, batch_size=25)
predictions, labels = utils.validate(model, test_loader, loss_function,
                                     device, step=0, tb_logger=None)

In [ ]:
import matplotlib.pyplot as plt
print("Test accuracy:")
accuracy = metrics.accuracy_score(labels, predictions)
print(accuracy)

fig, ax = plt.subplots(1, figsize=(8, 8))
utils.make_confusion_matrix(labels, predictions, categories, ax)

## Tasks and Questions

Tasks
- Try 3 different learning rate values and observe how this changes the training curves, training time and results on the test data.
- Try different architectures for your MLP. For example, you can increase the number of layers or use a different activation function (e.g. Sigmoid) and compare them on the test dataset.

Question:
- Which of the models you have trained in this and in the previous notebook perform best?
- Besides the final performance, how do the training curves differ?
- Can you find any systematic similarities or differences in the confusion matrix for the different models?
- What can you do to make the results of these comparisons more reliable?

Advanced:
- Try one or more of your architectures with preset filters (see last notebook) as input features.